## Import and Utils

In [ ]:
!pip install bayesian_optimization --quiet
!pip install ppscore --quiet
!pip install -q catboost
!pip install -q prince

%cd drive/MyDrive/he_public_griev

     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 65.8MB 43kB/s 
/content/drive/MyDrive/he_public_griev


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import prince
import string
from scipy import stats
from matplotlib import style
style.use('dark_background')
import ppscore as pps
#REgressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier, AdaBoostClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier

from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier, Dataset
from sklearn.ensemble import StackingClassifier
#Evalution Metrix
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.model_selection import KFold, GridSearchCV, cross_val_score, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, mean_squared_error, mean_squared_log_error

import warnings
warnings.filterwarnings('ignore')

def pd_full():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    #pd.set_option('display.width', 2000)
    #pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)

def pd_reset():
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [ ]:
def dfinfo(df, download="no",name=None):
    print("Dataset Shape", df.shape)
    tempdf = pd.DataFrame(df.dtypes, columns=['dtypes'])
    tempdf = tempdf.reset_index()
    tempdf.columns = ['Name','DType']
    tempdf['Missing'] = df.isnull().sum().values
    tempdf['Uniques'] = df.nunique().values
    tempdf['Missing %'] = ((df.isnull().sum().values)/ (df.shape[0])) *100
    #tempdf['skew'] = df.skew()
    tempdf['First Value'] = df.iloc[0].values
    tempdf['Second Value'] = df.iloc[1].values
    tempdf['Third Value'] = df.iloc[2].values

    for name in tempdf['Name'].value_counts().index:
        tempdf.loc[tempdf['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 
    
    if download.lower() == "yes":
        tempdf.to_csv('dataset_profile.csv', index=False)
        from google.colab import files
        files.download('dataset_profile.csv')

    return tempdf

In [ ]:
festive = '''1 Jan 2019
21 Mar 2019
15 Aug 2019
08 Oct 2019
27 Oct 2019
25 Dec 2019
29 Dec 2020
1 Jan 2020
10 Mar 2020
15 Aug 2020
25 Oct 2020
14 Nov 2020
25 Dec 2020
29 Dec 2020
'''

import datetime 

# t['Date'].to_pydatetime().date()
# t[['Date','Holiday']][t['Holiday']==True].head()

festive_list = []
range_list = []
for date in festive.splitlines():
    day, month_year = date.split(' ',1)
    range_list = [str(day)+' '+month_year for day in range(int(day)-2,int(day)+1) if int(day)>2]
    for date_range in range_list:
        date = datetime.datetime.strptime(date_range, '%d %b %Y') 
        if date.date() not in festive_list:
            festive_list.append(date.date())
            
def date_features(t, datecol):
    t[datecol+'Year'] = t[datecol].dt.year
    t[datecol+'Month'] = t[datecol].dt.month
    t[datecol+'Day_of_week'] = t[datecol].dt.dayofweek
    t[datecol+'_Day'] = t[datecol].dt.day
    t[datecol+'_DayofYear'] = t[datecol].dt.dayofyear
    t[datecol+'Week'] = pd.to_datetime(t[datecol]).dt.week 
    t[datecol+'Quarter'] = pd.to_datetime(t[datecol]).dt.quarter
    t[datecol+'Is_year_start'] = t[datecol].dt.is_year_start
    t[datecol+'Is_year_end'] = t[datecol].dt.is_year_end
    t[datecol+'Is_month_start'] = np.where(pd.to_datetime(t[datecol]).dt.is_month_start,1,0) 
    t[datecol+'Is_month_end'] = np.where(pd.to_datetime(t[datecol]).dt.is_month_end,1,0)
    t[datecol+'Is_weekend'] = np.where(t[datecol+'Day_of_week'].isin([5,6]),1,0)
    #t[datecol+'Is_Holiday'] = np.where(t[datecol].isin(festive_list),1,0)
    del t[datecol]
    return t 

def top_k_encoding(col,k):
   topklist = t[col].value_counts().index[:k].tolist() 
   for ver in topklist:
       t[col] = [ver if ver in ele else "other" for ele in t[col]]

def client_agent_proc(x, agt):
    for i in range(len(x)):
        if agt in x[i]:
            return x[i]

# t['client_agent'].apply(type).value_counts()


# def device_details(x):
#     if 'iPhone' in x:
#         return 'IPhone'
#     elif 'Android' in x:
#         return 'Android'
#     elif 'Desktop' in x:
#         return 'Desktop'
#     elif 'iPad' in x:
#         return 'iPad'
#     else:
#         return 'Other'

# from collections import Counter
# import string
# import re
# device_gran = t["client_agent"].iloc[0].lower().translate(str.maketrans('', '', string.punctuation)).split()
# t['client_agent'].str.split(expand=True).stack().value_counts()
# d = pd.DataFrame(t["client_agent"].str.lower().str.translate(str.maketrans('', '', string.punctuation)).str.split(' ',-1, expand=True))
# t["client_agent"].str.lower().str.translate(str.maketrans('', '', string.punctuation)).str.split().iloc[:20]
# t["client_agent"].str.lower().str.translate(str.maketrans('', '', string.punctuation)).str.split()

def client_agent(x):
    if 'iPhone' in str(x):
            return 'IPhone'
    elif 'Windows' in str(x):
        return 'Windows'
    elif 'Mac OS X' in str(x):
        return 'Mac'
    elif 'Android' in str(x):
        return 'Android'
    elif 'Linux' in str(x):
        return 'Linux'
    else:
        return 'Other'

# def mergedevice(x):
#     if x['client_agent']!=x['device_details']:
#         return x['client_agent']+x['device_details']
#     return x['client_agent']



def rmsle_lgbm(y_pred, data):

    y_true = np.array(data.get_label())
    score = np.sqrt(np.mean(np.power(np.log1p(y_true) - np.log1p(y_pred), 2)))

    return 'rmsle', score, False

## EDA

In [ ]:
df_train = pd.read_csv('dataset/train.csv')
df_test = pd.read_csv('dataset/test.csv')
sub = pd.read_csv('dataset/sample_submission.csv')
pd_full()

In [ ]:
df_train.head()

,appno,application,country.alpha2,country.name,decisiondate,docname,doctypebranch,ecli,introductiondate,issue.0,issue.1,issue.10,issue.11,issue.12,issue.13,issue.14,issue.15,issue.16,issue.17,issue.18,issue.19,issue.2,issue.20,issue.21,issue.22,issue.23,issue.24,issue.25,issue.26,issue.3,issue.4,issue.5,issue.6,issue.7,issue.8,issue.9,itemid,judgementdate,kpdate,languageisocode,originatingbody,originatingbody_name,originatingbody_type,parties.0,parties.1,parties.2,rank,respondent.0,respondent.1,respondent.2,respondent.3,respondent.4,respondentOrderEng,separateopinion,sharepointid,typedescription,article=3,article=6,article=P1,article=5,article=8,article=13,article=10,article=2,article=34,article=41,article=38,article=35,article=4,article=29,article=14,article=P12,article=9,article=1,article=11,article=46,article=P4,article=17,article=18,article=7,article=,article=37,article=P7,article=57,article=26,article=27,article=36,article=30,article=12,article=56,article=39,article=15,article=25,article=P6,article=19,article=32,article=33,article=53,article=28,article=43,article=52,article=P13,article=16,documentcollectionid=CASELAW,documentcollectionid=JUDGMENTS,documentcollectionid=CHAMBER,documentcollectionid=ENG,documentcollectionid=COMMITTEE,documentcollectionid=GRANDCHAMBER,applicability=,applicability=36,applicability=43,applicability=41,applicability=55,applicability=3,applicability=22,applicability=60,applicability=58,applicability=25,applicability=47,applicability=12,applicability=38,applicability=20,applicability=18,applicability=24,applicability=62,applicability=21,applicability=23,applicability=8,applicability=26,applicability=53,applicability=15,applicability=48,applicability=14,applicability=51,applicability=13,applicability=5,applicability=7,applicability=50,applicability=52,applicability=28,applicability=29,applicability=31,applicability=6,applicability=81,applicability=66,applicability=49,applicability=33,applicability=63,applicability=68,applicability=46,applicability=19,applicability=40,applicability=17,applicability=32,applicability=72,applicability=34,applicability=35,applicability=54,applicability=27,applicability=16,applicability=64,applicability=57,applicability=56,applicability=2,applicability=4,applicability=67,applicability=77,applicability=71,applicability=59,paragraphs=3,paragraphs=6,paragraphs=6-1,paragraphs=6-3-c,paragraphs=P1-3,paragraphs=5,paragraphs=5-1,paragraphs=P1-1,paragraphs=8,paragraphs=8-1,paragraphs=13,paragraphs=10,paragraphs=10-2,paragraphs=10-1,paragraphs=35-1,paragraphs=2,paragraphs=2-1,paragraphs=34,paragraphs=41,paragraphs=38,paragraphs=35,paragraphs=8-2,paragraphs=35-3-a,paragraphs=29-3,paragraphs=4,paragraphs=29,paragraphs=P1-1-1,paragraphs=5-1-e,paragraphs=6-2,paragraphs=5-1-c,paragraphs=14,paragraphs=35-3,paragraphs=6-3-d,paragraphs=5-1-b,paragraphs=5-5,paragraphs=P12-1,paragraphs=11-1,paragraphs=11-2,paragraphs=9,paragraphs=9-1,paragraphs=1,paragraphs=11,paragraphs=6-3,paragraphs=5-3,paragraphs=5-4,paragraphs=46,paragraphs=46-2,paragraphs=5-1-a,paragraphs=P4-2,paragraphs=P4-2-2,paragraphs=17,paragraphs=18,paragraphs=7,paragraphs=7-1,paragraphs=,paragraphs=4-3-d,paragraphs=4-2,paragraphs=6-3-e,paragraphs=9-2,paragraphs=P1-1-2,paragraphs=P4-2-1,paragraphs=37-1-c,paragraphs=37,paragraphs=P7-4,paragraphs=4-1,paragraphs=57,paragraphs=6-3-b,paragraphs=35-3-b,paragraphs=5-1-f,paragraphs=35-4,paragraphs=P4-2-3,paragraphs=2-2,paragraphs=26,paragraphs=27-2,paragraphs=27,paragraphs=36,paragraphs=36-1,paragraphs=37-1,paragraphs=P7-2,paragraphs=35-2,paragraphs=5-1-d,paragraphs=6-3-a,paragraphs=30,paragraphs=46-1,paragraphs=12,paragraphs=P7-1,paragraphs=P7-1-1,paragraphs=36-2,paragraphs=5-2,paragraphs=P1-2,paragraphs=P1-4,paragraphs=56-3,paragraphs=56,paragraphs=56-1,paragraphs=P7-1-2,paragraphs=38-1-a,paragraphs=37-1-b,paragraphs=39,paragraphs=15-1,paragraphs=15,paragraphs=7-2,paragraphs=P4-4,paragraphs=15-3,paragraphs=25-1,paragraphs=25,paragraphs=37-1-a,paragraphs=P6-1,paragraphs=P7-5,paragraphs

In [ ]:
df_train = df_train[df_train['parties.1'].notna()]

In [ ]:
t = pd.concat([df_train.drop('importance', axis=1), df_test], ignore_index=True)

In [ ]:
for col in t.columns:
    if t[col].isnull().sum() / t.shape[0] > 0.6 or t[col].nunique() > 7000 or t[col].nunique() == 1:
        t.drop(col, axis=1, inplace=True)

In [ ]:
dfinfo(t)

Dataset Shape (13634, 281)


,Name,DType,Missing,Uniques,Missing %,First Value,Second Value,Third Value,Entropy
0,country.alpha2,object,0,46,0.000000,ru,tr,si,4.50
1,country.name,object,0,46,0.000000,Russian Federation,Turkey,Slovenia,4.50
2,doctypebranch,object,0,3,0.000000,CHAMBER,COMMITTEE,CHAMBER,0.90
3,judgementdate,object,0,2087,0.000000,28/05/2009,10/07/2018,18/10/2012,10.51
4,kpdate,object,0,2087,0.000000,28/05/2009,10/07/2018,18/10/2012,10.51
5,originatingbody,int64,0,13,0.000000,4,26,23,3.27
6,originatingbody_name,object,0,13,0.000000,First Section,Second Section Committee,Fith Section,3.27
7,parties.1,object,2,107,0.014669,RUSSIA,TURKEY,SLOVENIA,4.61
8,rank,float64,0,6482,0.000000,685.417,2641.9,942.213,12.26
9,respondent.0,object,0,46,0.000000,RUS,TUR,SVN,4.50


In [ ]:
t['judgementdate'] = pd.to_datetime(t['judgementdate'])
t['kpdate'] = pd.to_datetime(t['kpdate'])

In [ ]:
date_features(t, 'judgementdate')
date_features(t, 'kpdate')

In [ ]:
for col in t.columns:
    print("Column : ", col, "\n", t[col].value_counts(),"\n")

In [ ]:
sns.displot(t['rank']);

## Model

In [ ]:
cat_cols = [t.columns.get_loc(col) for col in t.columns if t[col].nunique() < 150]
# [trn.columns.get_loc(c) for c in cat_cols]

In [ ]:
trn = t[:df_train.shape[0]]
tst = t[df_train.shape[0]:]
tst['parties.1'].fillna("None", inplace=True)
y = df_train.importance

trn = trn.reset_index(drop=True)
tst = tst.reset_index(drop=True)
y = y.reset_index(drop=True)

print(trn.shape, y.shape, tst.shape)

In [ ]:
baseline_models = {
    # "SVC" : SVC(),
    # "rfclf" : RandomForestClassifier(n_jobs=-1),
    # "lgb" : LGBMClassifier(),
    # 'xgb' : XGBClassifier(),
    # 'extratrees' : ExtraTreesClassifier(n_jobs=-1),
    # 'DTC' : DecisionTreeClassifier(),
    "cb" : CatBoostClassifier(cat_features = cat_cols, verbose=False, task_type="GPU", devices='0:1'),
    # "gbc" : GradientBoostingClassifier(random_state=0),
}

In [ ]:
trn.head()

In [ ]:
def kfold_cv(trn, target, estimator_name, estimator, n_splits=5, shuffle_val=False, featimp=False):
    featimp_list = []
    add = 0
    splits = n_splits
    if shuffle_val:
        kfold, scores = StratifiedKFold(n_splits=splits,shuffle=shuffle_val), list()
    else:
        kfold, scores = StratifiedKFold(n_splits=splits, shuffle=shuffle_val, random_state=42), list()
    for train, test in kfold.split(trn,target):
        x_train, x_test = trn.values[train], trn.values[test]
        y_train, y_test = target.values[train], target.values[test]
        model = estimator
        model.fit(x_train, y_train)
        preds = model.predict(x_test)

        if featimp:
            feature_importances = pd.DataFrame(model.feature_importances_,
                                            index = trn.columns,
                                            columns=['importance'])
            featimp_list.append(feature_importances.values)

        score = accuracy_score(y_test, preds)
        scores.append(score)
        print("Accuracy Score is", score)

    print(f"Average of model {estimator_name} is {np.sum(scores)/len(scores)} and per fold scores are {scores}")
    if featimp:
        for item in featimp_list:
            add+=item
        df_cv=pd.DataFrame(add/len(featimp_list),index=trn.columns,columns=["importance"]).sort_values('importance', ascending=False)
        display(df_cv)
    return estimator_name, np.sum(scores)/len(scores)

In [ ]:
best_models = {}
for model_name, model in baseline_models.items():
    if model_name in ['lgb','xgb','cb']:
        estname, cvscore = kfold_cv(trn, y, model_name, model, n_splits=5, shuffle_val=True, featimp=True)
    else:
        estname, cvscore = kfold_cv(trn, y, model_name, model, n_splits=5, shuffle_val=True)
    best_models[estname] = cvscore

Accuracy Score is 0.88
Accuracy Score is 0.887887323943662
Accuracy Score is 0.864225352112676
Accuracy Score is 0.8850704225352113
Accuracy Score is 0.8754227733934611
Average of model cb is 0.8785211743970021 and per fold scores are [0.88, 0.887887323943662, 0.864225352112676, 0.8850704225352113, 0.8754227733934611]


,importance
rank,1.457843e+01
paragraphs=41,1.223747e+01
article=41,1.076914e+01
judgementdateYear,3.745397e+00
parties.1,3.613779e+00
article=35,3.259579e+00
ccl_article=6,3.076791e+00
paragraphs=35,2.957651e+00
separateopinion,2.893895e+00
originatingbody_name,2.690859e+00


## Inference

In [ ]:
preds = model.predict(tst.values)

In [ ]:
np.array(np.unique(preds, return_counts=True)).T

array([[   1,  264],
       [   2,  168],
       [   3, 1122],
       [   4, 3206]])

In [ ]:
subm = pd.DataFrame(columns=['appno',"importance"])
subm.appno = df_test.appno
subm.importance = preds

In [ ]:
subm.head()

,appno,importance
0,1194/04,4
1,53865/11,4
2,43838/07,4
3,11157/04,3
4,42443/02,3


In [ ]:
subm.to_csv('base_cb.csv', index=False)
from google.colab import files
files.download("base_cb.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>